# MAP-Elites track generations

In [ ]:
import numpy as np
import requests
import random
import matplotlib.pyplot as plt
import joblib    
# Just load a saved t-SNE or UMAP model     
EMBEDDING_MODEL = joblib.load("embedding/umap_model.joblib")   # 2-D model trained offline


from ribs.archives import SlidingBoundariesArchive
from ribs.emitters import EmitterBase
from ribs.schedulers import Scheduler
from dask.distributed import Client, LocalCluster, as_completed
from ribs.visualize import sliding_boundaries_archive_heatmap


In [ ]:
BASE_URL = 'http://localhost:4242'
GENERATION_MODE = 'voronoi' #'convexHull' or 'voronoi'
POINTS_COUNT = 100
MAX_SELECTED_CELLS = 10 # relevant only for voronoi
SOLUTION_DIM = POINTS_COUNT * 2 + MAX_SELECTED_CELLS * 2 + 1 
TRACK_SIZE_RANGE = (4,10) #(4, 10) for voronoi otherwise (100,100)
ITERATIONS = 1000
ARCHIVE_DIM = 10
INIT_POPULATION = ARCHIVE_DIM * ARCHIVE_DIM 

BATCH_SIZE=10

INVALID_SCORE    = -1e9
CHECKPOINT_EVERY = 50

DEBUG_CROSSOVER = True
DEBUG_MUTATION = True


ARCHIVE_BINS = 30                               # cells per axis
REMAPPING_EVERY = 200                           # move boundaries every 200 insertions
BUFFER_SIZE = 1000                              # keep last 1000 solutions

In [ ]:
cluster = LocalCluster(processes=True, n_workers=BATCH_SIZE, threads_per_worker=1)
client = Client(cluster)

### Helper functions

In [ ]:
def generate_solution(iteration):
    print(f"Generating solution for iteration {iteration}")
    try:
        response = requests.post(
            f"{BASE_URL}/generate",
            json={
                "id": iteration + random.random(),
                "mode": GENERATION_MODE,
                "trackSize": random.randint(TRACK_SIZE_RANGE[0], TRACK_SIZE_RANGE[1])
            },
            timeout=60
        )
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error generating solution for iteration {iteration}: {e}")
        return None

def solution_to_array(sol):
    if sol is None:
        return None
    arr = np.zeros(SOLUTION_DIM)
    for i, p in enumerate(sol.get("dataSet", [])):
        arr[i * 2] = p.get("x", 0)
        arr[i * 2 + 1] = p.get("y", 0)
    for i, c in enumerate(sol.get("selectedCells", [])):
        if i < MAX_SELECTED_CELLS:
            idx = POINTS_COUNT * 2 + i * 2
            arr[idx] = c.get("x", 0)
            arr[idx + 1] = c.get("y", 0)
    arr[-1] = sol.get("id", 0)
    return arr

def array_to_solution(arr):
    ds = []
    for i in range(0, POINTS_COUNT * 2, 2):
        ds.append({"x": float(arr[i]), "y": float(arr[i+1])})
    sel = []
    for i in range(POINTS_COUNT * 2, SOLUTION_DIM - 1, 2):
        x_val = arr[i]
        y_val = arr[i+1]
        if x_val != 0 or y_val != 0:
            sel.append({"x": float(x_val), "y": float(y_val)})
    return {
        "id": float(arr[-1]),
        "mode": GENERATION_MODE,
        "dataSet": ds,
        "selectedCells": sel
    }

def get_fractional_part(x):
    return x - int(x)

def pca_align(points):
    pts = points - points.mean(0)
    u, _, _ = np.linalg.svd(pts, full_matrices=False)
    angle = np.arctan2(u[1, 0], u[0, 0])
    rot = np.array([[np.cos(-angle), -np.sin(-angle)],
                    [np.sin(-angle),  np.cos(-angle)]])
    aligned = pts @ rot.T
    if aligned[0, 0] < 0:
        aligned[:, 0] *= -1
    return aligned

def descriptor_from_track(sol):
    pts = np.array([[p["x"], p["y"]] for p in sol.get("splineVector", [])], dtype=float)
    aligned = pca_align(pts)
    flat = aligned.ravel()
    return EMBEDDING_MODEL.transform(flat[None, :])[0]

_STATS = {
    'len_min':  np.inf, 'len_max':  -np.inf,
    'ov_min':   np.inf, 'ov_max':   -np.inf,
    'dx_min':   np.inf, 'dx_max':   -np.inf,
    'bend_min': np.inf, 'bend_max': -np.inf,
}

def _upd(k, v):
    lo, hi = f'{k}_min', f'{k}_max'
    if v < _STATS[lo]: _STATS[lo] = v
    if v > _STATS[hi]: _STATS[hi] = v

def _norm(k, v, eps=1e-6):
    lo, hi = _STATS[f'{k}_min'], _STATS[f'{k}_max']
    return (v - lo) / (hi - lo + eps)

def fitness_formula(fit):
    length     = max(fit.get('length', 0.0), 1e-3)
    bend_len   = fit.get('right_len', 0.0) + fit.get('left_len', 0.0)
    overtakes  = fit.get('total_overtakes', 0.0)
    dx         = abs(fit.get('deltaX', 0.0)) or 1e-3

    bend_ratio = bend_len / length

    for k, v in (('len', length), ('ov', overtakes), ('dx', dx),
                 ('bend', bend_ratio)):
        _upd(k, v)

    score = (
        0.25 * _norm('len',  length)         +  # encourage longer tracks
        0.60 * _norm('bend', bend_ratio)     +  # maximise curves per metre
        0.15 * (_norm('ov',  overtakes) /
                (_norm('dx', dx) + 1e-3))       # overtakes, damped by Δx
    )
    return float(score)


def evaluate_solution(sol):
    sol_id = sol.get("id", 0)
    ok = True
    msg = ""
    try:
        r = requests.post(f"{BASE_URL}/evaluate", json=sol, timeout=60)
        r.raise_for_status()
        r_json = r.json()
        fit = r_json.get("fitness", {})
        desc = descriptor_from_track(r_json)  
    except Exception as e:
        ok = False 
        msg = str(e)
        desc = np.zeros((2,))  # just in case of error
        fit = INVALID_SCORE

    return sol_id, ok, msg, fit, desc



## Genetic operators

In [ ]:
class CustomEmitter(EmitterBase):
    def __init__(self, archive, solution_dim, batch_size=BATCH_SIZE, bounds=None):
        super().__init__(archive, solution_dim=solution_dim, bounds=bounds)
        self.batch_size = batch_size
        self.iteration = 0

    def ask(self):
        self.iteration += 1
        print(f"Emitter.ask() called for iteration {self.iteration}")
        if self.iteration <= INIT_POPULATION:
            out = []
            for _ in range(self.batch_size):
                sol = generate_solution(self.iteration - 1)
                arr = solution_to_array(sol)
                if arr is not None:
                    out.append(arr)
                else:
                    out.append(np.full(SOLUTION_DIM, INVALID_SCORE))
            return np.array(out)
        else:
            if random.random() < 0.5:
                return self.mutate_solutions()
            else:
                return self.crossover_solutions()

    def mutate_solutions(self):
        print(f"Mutating solutions for iteration {self.iteration}")
        parents = self.archive.sample_elites(self.batch_size)
        out = []
        for i in range(self.batch_size):
            arr = parents["solution"][i]
            sol = array_to_solution(arr)
            try:
                response = requests.post(
                    f"{BASE_URL}/mutate",
                    json={
                        "individual": sol,
                        "intensityMutation": 20
                    },
                    timeout=60
                )
                response.raise_for_status()
                mutated = response.json().get("mutated", {})
                frac = get_fractional_part(sol["id"])
                mutated["id"] = self.iteration - 1 + frac
                mutated_arr = solution_to_array(mutated)
                if mutated_arr is not None:
                    out.append(mutated_arr)
                    print(f"Mutated ID={sol['id']} to ID={mutated['id']}")
                else:
                    out.append(np.full(SOLUTION_DIM, INVALID_SCORE))
            except requests.RequestException as e:
                print(f"Error mutating solution ID={sol['id']}: {e}")
                out.append(np.full(SOLUTION_DIM, INVALID_SCORE))
        return np.array(out)

    def crossover_solutions(self):
        print(f"Crossover solutions for iteration {self.iteration}")
        out = []
        for _ in range(self.batch_size // 2):
            try:
                while True:
                    parents = self.archive.sample_elites(2)
                    sol1 = array_to_solution(parents["solution"][0])
                    sol2 = array_to_solution(parents["solution"][1])
                    if sol1["id"] != sol2["id"]:
                        break
                response = requests.post(
                    f"{BASE_URL}/crossover",
                    json={
                        "mode": GENERATION_MODE,
                        "parent1": sol1,
                        "parent2": sol2
                    },
                    timeout=60
                )
                response.raise_for_status()
                offspring = response.json().get("offspring", {})
                f1 = get_fractional_part(sol1["id"])
                f2 = get_fractional_part(sol2["id"])
                frac = (f1 + f2) % 1
                child_id = self.iteration - 1 + frac
                child_sol = {
                    "id": child_id,
                    "mode": GENERATION_MODE,
                    "trackSize": len(offspring.get("sel", [])),
                    "dataSet": offspring.get("ds", []),
                    "selectedCells": offspring.get("sel", [])
                }
                child_arr = solution_to_array(child_sol)
                if child_arr is not None:
                    out.append(child_arr)
                    print(f"Crossover Parent1 ID={sol1['id']}, Parent2 ID={sol2['id']} => Child ID={child_id}")
                else:
                    out.append(np.full(SOLUTION_DIM, INVALID_SCORE))
            except requests.RequestException as e:
                print(f"Error during crossover: {e}")
                out.append(np.full(SOLUTION_DIM, INVALID_SCORE))
        return np.array(out)

## Illuminating search spaces by mapping elites


In [ ]:
import glob
import os
import pickle

archive = SlidingBoundariesArchive(
    solution_dim=SOLUTION_DIM,
    dims=[ARCHIVE_BINS, ARCHIVE_BINS],
    ranges=[(-1, 1), (-1, 1)],
    remap_frequency=REMAPPING_EVERY,
    buffer_capacity=BUFFER_SIZE
)

emitter = CustomEmitter(
    archive,
    solution_dim=SOLUTION_DIM,
    batch_size=BATCH_SIZE,
    bounds=[(0, 600)] * (SOLUTION_DIM - 1) + [(0, float("inf"))]
)

scheduler = Scheduler(archive, [emitter])

# ──────────────────────────────────────────────────────────────
# Resume from latest pickle checkpoint if one exists
# ──────────────────────────────────────────────────────────────
checkpoints = sorted(glob.glob("checkpoint_*.pkl"))
start_iter = 0
global_best_score = INVALID_SCORE
global_best_id = None

if checkpoints:
    latest_ckpt = checkpoints[-1]
    with open(latest_ckpt, "rb") as f:
        state = pickle.load(f)
    scheduler           = state["scheduler"]
    archive  = scheduler.archive
    start_iter          = state["iteration"]
    global_best_score   = state["global_best_score"]
    global_best_id      = state["global_best_id"]
    print(f"[Resume] Loaded {latest_ckpt}, resuming from iteration {start_iter+1}")
else:
    print("[Resume] No checkpoint found — starting fresh.")


def run_map_elites(total_iters, start_iter=0):
    # declare that we mean to update these module‐level variables
    global global_best_score, global_best_id

    for i in range(start_iter, total_iters):
        print(f"=== Starting iteration {i+1} ===")
        sols      = scheduler.ask()
        sol_dicts = [array_to_solution(s) for s in sols]
        futs      = [client.submit(evaluate_solution, sol) for sol in sol_dicts]
        gathered  = [f.result() for f in as_completed(futs)]

        obj_list, clean = [], []
        for sol_id, ok, msg, fit, desc in gathered:
            score = INVALID_SCORE
            if not ok:
                print(f"Warning: clamping bad score for ID={sol_id} ({msg})")
            else:
                score = fitness_formula(fit)
                print(f"Solution ID={sol_id} evaluated with score={score:.2f}")
                if score > global_best_score:
                    global_best_score = score
                    global_best_id    = sol_id
            clean.append((score, desc))
            obj_list.append(score)

        obj_batch, meas_batch = zip(*clean)
        scheduler.tell(list(obj_batch), list(meas_batch))

        batch_best = max(obj_list) if obj_list else INVALID_SCORE
        print(f"Iteration {i+1} ended. Best in batch = {batch_best:.2f}")
        print(f"Global best so far: {global_best_score:.2f} (ID={global_best_id})")

        # ── Stats ──
        data = archive.data()
        if data:
            arch_obj = data["objective"]
            valid    = arch_obj != INVALID_SCORE
            mean_val = np.mean(arch_obj[valid]) if np.any(valid) else 0.0
            best_val = np.max(arch_obj[valid]) if np.any(valid) else 0.0
            cov      = archive.stats.coverage
            print(f"Archive size={len(archive)}, cov={cov:.3f}, mean={mean_val:.2f}, best={best_val:.2f}")
        else:
            print("Archive still empty")

        # ── Checkpoint ──
        if (i + 1) % 50 == 0:
            ckpt_name = f"checkpoint_{i+1:04d}.pkl"
            with open(ckpt_name, "wb") as f:
                pickle.dump({
                    "scheduler":         scheduler,
                    "iteration":         i+1,
                    "global_best_score": global_best_score,
                    "global_best_id":    global_best_id
                }, f)
            print(f"[Checkpoint] Saved {ckpt_name}")

         # ── Plot every 5 iterations ─────────────────────────────────
        if (i + 1) % 10 == 0:
            # Pull objective array from archive data
            arch_obj = np.array(archive.data()["objective"])
            valid    = arch_obj != INVALID_SCORE

            if np.any(valid):
                # Compute percentiles to clip outliers
                p_low, p_high = np.percentile(arch_obj[valid], [20, 80])
            else:
                p_low = p_high = 0.0

            fig = plt.figure(figsize=(8, 6))
            sliding_boundaries_archive_heatmap(
                archive,
                boundary_lw=0.5,
                vmin=p_low,
                vmax=p_high,
            )
            plt.title(f"Archive Heat-map – Iteration {i+1}")
            plt.xlabel("UMAP-1")
            plt.ylabel("UMAP-2")
            plt.tight_layout()
            plt.savefig(f"archive_heatmap_iter_{i+1}.png")
            plt.close(fig)

In [ ]:
run_map_elites(ITERATIONS, start_iter)

# Visualize results

In [ ]:
print(f"Final archive size={len(archive)}, Coverage={archive.stats.coverage:.3f}")
size      = len(archive)
coverage  = archive.stats.coverage
objs      = archive.data()["objective"]
data      = archive.data()

print(f"Archive size     : {size}")
print(f"Coverage         : {coverage:.3%}")
print(f"Best objective   : {objs.max():.3f}")
print(f"Mean objective   : {objs.mean():.3f}\n")

k = 10                                        # how many to list
for i in range(min(k, size)):
    sol_vec = data["solution"][i]
    sol_id  = sol_vec[-1]                     # float ID
    umap_xy = data["measures"][i]
    print(f"{i+1:2d}. "
          f"ID={sol_id:.17f}   "
          f"UMAP=({umap_xy[0]:7.3f}, {umap_xy[1]:7.3f})")




In [ ]:
target_x = 10.0   # change this as needed
k        = 4     # how many neighbours to display

measures = data["measures"]                    # (N, 2) array
dx       = np.abs(measures[:, 0] - target_x)   # distance to target on x-axis

nearest_idx = np.argsort(dx)[:k]               # indices of the k closest
for rank, idx in enumerate(nearest_idx, 1):
    sol_vec = data["solution"][idx]
    sol_id  = sol_vec[-1]
    umap_xy = measures[idx]
    score   = objs[idx]
    print(f"{rank:2d}. "
          f"ID={sol_id:.17f}   "
          f"dx={dx[idx]:7.3f}   "
          f"score={score:8.3f}   "
          f"UMAP=({umap_xy[0]:7.3f}, {umap_xy[1]:7.3f})")
    
measures   = data["measures"]            # (N, 2) array of UMAP coordinates
all_x, all_y = measures[:, 0], measures[:, 1]

# colour entire archive by objective score
plt.figure(figsize=(8, 6))
sc = plt.scatter(all_x, all_y,
                 c=objs,                # colour scale = performance
                 s=12,                  # dot size
                 alpha=0.8,
                 cmap="viridis")

# highlight the nearest k in red with black edge
highlight = measures[nearest_idx]
plt.scatter(highlight[:, 0], highlight[:, 1],
            facecolors='none',
            edgecolors='red',
            s=80,
            linewidths=1.5,
            label=f"{len(nearest_idx)} closest to UMAP-x={target_x}")

plt.colorbar(sc, label="Objective value")
plt.title("Archive in UMAP space\n(highlighted = closest to target x)")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


In [ ]:
m = 15                                   # how many random elites to inspect
rng = np.random.default_rng(42)          # reproducible
rand_idx = rng.choice(len(archive), size=m, replace=False)

samples = []                             # keep the decoded solutions here
for j, idx in enumerate(rand_idx, 1):
    sol_vec = data["solution"][idx]
    sol_id  = sol_vec[-1]
    umap_xy = data["measures"][idx]
    score   = objs[idx]
    print(f"{j:2d}. ID={sol_id:.17f}   "
          f"score={score:8.3f}   "
          f"UMAP=({umap_xy[0]:7.3f}, {umap_xy[1]:7.3f})")
    
    # store full dict in `samples` for later, if desired
    samples.append(array_to_solution(sol_vec))


import matplotlib.pyplot as plt
import numpy as np

# --------------------------------------------------------------
#   filter out objective outliers (beyond ±3 σ from the mean)
# --------------------------------------------------------------
obj_mean = objs.mean()
obj_std  = objs.std()
lo, hi   = obj_mean - 3*obj_std, obj_mean + 3*obj_std
mask     = (objs >= lo) & (objs <= hi)

x_f, y_f = all_x[mask], all_y[mask]
w_f      = objs[mask]                       # colour = objective value

plt.figure(figsize=(8, 6))
hb = plt.hist2d(x_f, y_f,
                bins=50,
                weights=w_f,
                cmap="plasma",
                vmin=obj_mean - obj_std,    # <<< centre the colour bar
                vmax=obj_mean + obj_std)    #     on the mean range
plt.colorbar(hb[3], label="Mean objective in bin")
plt.title("Performance landscape (outliers clipped, scale ~ mean ±1 σ)")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.tight_layout()
plt.show()

